In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.6.0+cu118
True
NVIDIA GeForce RTX 3060 Ti


In [2]:
!pip install python-dotenv, transfor


In [3]:
from dotenv import load_dotenv
import os, torch
from transformers import AutoModel, AutoTokenizer


ModuleNotFoundError: No module named 'transformers'

In [20]:
model = AutoModel.from_pretrained("/home/tess/work/deep_learning/transformers/models/mobilebert/model")
tokenizer = AutoTokenizer.from_pretrained("/home/tess/work/deep_learning/transformers/models/mobilebert/tokenizer")
device =  "cuda" if torch.cuda.is_available() else "cpu"


In [23]:
from transformers import TextClassificationPipeline, MobileBertForSequenceClassification
# classifier = TextClassificationPipeline(model=model,tokenizer=tokenizer, device=device)
classifier = MobileBertForSequenceClassification.from_pretrained("/home/tess/work/deep_learning/transformers/models/mobilebert/model")

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at /home/tess/work/deep_learning/transformers/models/mobilebert/model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
sentence = "I love  you "
inputs = tokenizer(sentence, return_tensors="pt")
outputs = classifier(**inputs).logits

In [27]:
outputs

tensor([[13264546.0000,   765925.7500]], grad_fn=<AddmmBackward0>)

In [29]:
predicted_class_id = outputs.argmax().item()

In [30]:
predicted_class_id

0

# fine tuneing 

In [7]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
dataset.num_rows

{'train': 8530, 'validation': 1066, 'test': 1066}

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [9]:
from transformers import RobertaTokenizer
device =  "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = RobertaTokenizer.from_pretrained("/home/tess/work/deep_learning/transformers/models/roberta/tokenizer")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [10]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "/home/tess/work/deep_learning/transformers/models/roberta/model",
    config=config,
)

In [11]:
# Add a new adapter
model.add_adapter("rotten_tomatoes", config="seq_bn", 
                  set_active=True, overwrite_ok=True)
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    overwrite_ok=True,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("rotten_tomatoes")

In [14]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [15]:
trainer.train()

Step,Training Loss
200,0.469000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=267, training_loss=0.43841320566470254, metrics={'train_runtime': 236.4749, 'train_samples_per_second': 36.071, 'train_steps_per_second': 1.129, 'total_flos': 359388592699200.0, 'train_loss': 0.43841320566470254, 'epoch': 1.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.3238898813724518,
 'eval_acc': 0.8658536585365854,
 'eval_runtime': 15.4984,
 'eval_samples_per_second': 68.781,
 'eval_steps_per_second': 2.194,
 'epoch': 1.0}

In [23]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("I wanna rim your pussy")

Device set to use cuda:0
The model 'RobertaAdapterModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gemma2

[{'label': '👎', 'score': 0.5504029989242554}]

In [27]:
model.save_adapter("./adapters/roberta", "rotten_tomatoes")
